In [2]:
import  tensorflow as tf
from tensorflow.keras import layers
import numpy as np

2024-07-26 22:27:46.511681: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
dataset_dir = "PlantVillage"
data = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_dir,
    shuffle=True,
    image_size=(256,256),
    batch_size=32
)

Found 2152 files belonging to 3 classes.


In [4]:
class_names = data.class_names
class_names

['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

In [ ]:
#Visualization of images
import matplotlib.pyplot as plt
plt.figure(figsize=(12,8))
for img,label in data.take(1):
    for i in range(15):
        ax = plt.subplot(3,5,i+1)
        plt.imshow(img[i].numpy().astype("uint8"))
        plt.title(class_names[label[i]])
        plt.axis('off')    
    plt.show()

In [5]:
import math
data.shuffle(10000)#to shuffle images with labels
print("Size of dataset = ", len(data))
train_size = math.floor(len(data)*0.8)
train_ds = data.take(train_size)
print("size of train dataset = ",len(train_ds))

Size of dataset =  68
size of train dataset =  54


In [6]:
test_ds = data.skip(train_size)
print("size of test dataset = ",len(test_ds))

size of test dataset =  14


In [7]:
val_ds = test_ds.take(6)
print("size of validation dataset = ",len(val_ds))

size of validation dataset =  6


In [8]:
test_ds = test_ds.skip(6)
print("size of test dataset = ",len(test_ds))

size of test dataset =  8


In [9]:
#Autotune data
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [10]:
input_shape = (32,256,256,3)
n_classes = 3

In [11]:
model = tf.keras.models.Sequential(
    [layers.Resizing(256,256),
     layers.Rescaling(1.0/255),
     layers.RandomFlip("horizontal_and_vertical"),
     layers.RandomRotation(0.2),
     layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=input_shape),
     layers.MaxPooling2D((2, 2)),
     layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
     layers.MaxPooling2D((2, 2)),
     layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
     layers.MaxPooling2D((2, 2)),
     layers.Conv2D(64, (3, 3), activation='relu'),
     layers.MaxPooling2D((2, 2)),
     layers.Conv2D(64, (3, 3), activation='relu'),
     layers.MaxPooling2D((2, 2)),
     layers.Conv2D(64, (3, 3), activation='relu'),
     layers.MaxPooling2D((2, 2)),
     layers.Flatten(),
     layers.Dense(64, activation='relu'),
     layers.Dense(n_classes, activation='softmax'),
    ])
model.build(input_shape=input_shape)

/home/ansh/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resizing (Resizing)             │ (32, 256, 256, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (32, 256, 256, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_flip (RandomFlip)        │ (32, 256, 256, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_rotation                 │ (32, 256, 256, 3)      │             0 │
│ (RandomRotation)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (32, 254, 254, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (32, 127, 127, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (32, 125, 125, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (32, 62, 62, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (32, 60, 60, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (32, 30, 30, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (32, 28, 28, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (32, 14, 14, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (32, 12, 12, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (32, 6, 6, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (32, 4, 4, 64)         │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (32, 2, 2, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (32, 256)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (32, 64)               │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (32, 3)                │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 183,747 (717.76 KB)

 Trainable params: 183,747 (717.76 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [14]:
history = model.fit(
    train_ds,
    batch_size=32,
    validation_data=val_ds,
    verbose=1,
    epochs=15,
)

Epoch 1/15


2024-07-26 22:28:28.559080: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:13: Filling up shuffle buffer (this may take a while): 53 of 1000
2024-07-26 22:28:28.681034: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.
2024-07-26 22:28:28.995728: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 264257536 exceeds 10% of free system memory.
2024-07-26 22:28:31.619095: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 128000000 exceeds 10% of free system memory.
2024-07-26 22:28:32.538236: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 128000000 exceeds 10% of free system memory.
2024-07-26 22:28:32.969674: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 132128768 exceeds 10% of free system memory.
2024-07-26 22:28:32.969985: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 264257536 exceeds 10% of fr

54/54 ━━━━━━━━━━━━━━━━━━━━ 108s 2s/step - accuracy: 0.5015 - loss: 0.9428 - val_accuracy: 0.6094 - val_loss: 0.8146
Epoch 2/15
54/54 ━━━━━━━━━━━━━━━━━━━━ 92s 2s/step - accuracy: 0.6690 - loss: 0.6966 - val_accuracy: 0.6771 - val_loss: 0.6296
Epoch 3/15
54/54 ━━━━━━━━━━━━━━━━━━━━ 91s 2s/step - accuracy: 0.7571 - loss: 0.5271 - val_accuracy: 0.8542 - val_loss: 0.4659
Epoch 4/15
54/54 ━━━━━━━━━━━━━━━━━━━━ 91s 2s/step - accuracy: 0.8294 - loss: 0.3886 - val_accuracy: 0.8958 - val_loss: 0.3164
Epoch 5/15
54/54 ━━━━━━━━━━━━━━━━━━━━ 91s 2s/step - accuracy: 0.8855 - loss: 0.2752 - val_accuracy: 0.8906 - val_loss: 0.2694
Epoch 6/15
54/54 ━━━━━━━━━━━━━━━━━━━━ 91s 2s/step - accuracy: 0.8932 - loss: 0.2567 - val_accuracy: 0.8750 - val_loss: 0.3584
Epoch 7/15
54/54 ━━━━━━━━━━━━━━━━━━━━ 91s 2s/step - accuracy: 0.8902 - loss: 0.3328 - val_accuracy: 0.9167 - val_loss: 0.2375
Epoch 8/15
54/54 ━━━━━━━━━━━━━━━━━━━━ 91s 2s/step - accuracy: 0.9064 - loss: 0.2479 - val_accuracy: 0.9010 - val_loss: 0.2351
Ep

In [15]:
model.evaluate(test_ds)

8/8 ━━━━━━━━━━━━━━━━━━━━ 12s 323ms/step - accuracy: 0.9647 - loss: 0.1665


[0.1540984809398651, 0.9612069129943848]

In [ ]:
def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(images[i].numpy())
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)

    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = round(100 * (np.max(predictions[0])), 2)
    return predicted_class, confidence

In [ ]:
#check prediction value
plt.figure(figsize=(15, 18))
for images, labels in test_ds.take(1):
    for i in range(20):
        ax = plt.subplot(5, 4, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        
        predicted_class, confidence = predict(model, images[i].numpy())
        actual_class = class_names[labels[i]] 
        
        plt.title(f"Actual: {actual_class},\n Predicted: {predicted_class}.\n Confidence: {confidence}%")
        
        plt.axis("off")

In [18]:
tf.keras.models.save_model(model,filepath="potato disease.keras")